<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/research/research/notebooks/riemann_zeros_toolkit_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Riemann Zeros Toolkit for GIFT Research

**Purpose**: Download, cache, and analyze Riemann zeta zeros for GIFT correspondence research.

**Features**:
- Bulk download from Odlyzko tables (2M+ zeros)
- Local caching for fast reload
- CuPy GPU acceleration for analysis
- mpmath integration for high-precision computation
- GIFT correspondence validation tools

**Requirements**: Colab Pro+ with A100 recommended for large-scale analysis.

In [1]:
# Install dependencies (run once)
!pip install -q mpmath requests tqdm

In [2]:
import os
import json
import requests
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from typing import Optional, Tuple, List
import warnings

# GPU support (optional)
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print(f"✓ CuPy available - GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
except ImportError:
    cp = np  # Fallback to NumPy
    GPU_AVAILABLE = False
    print("⚠ CuPy not available, using NumPy (CPU)")

# High precision (optional)
try:
    from mpmath import mp, zetazero, zeta
    mp.dps = 50  # 50 decimal places default
    MPMATH_AVAILABLE = True
    print(f"✓ mpmath available - precision: {mp.dps} digits")
except ImportError:
    MPMATH_AVAILABLE = False
    print("⚠ mpmath not available, high-precision computation disabled")

✓ CuPy available - GPU: NVIDIA A100-SXM4-80GB
✓ mpmath available - precision: 50 digits


## 1. Data Sources Configuration

In [3]:
# Odlyzko data sources
ODLYZKO_BASE = "https://www-users.cse.umn.edu/~odlyzko/zeta_tables"

ODLYZKO_FILES = {
    "zeros1": {
        "url": f"{ODLYZKO_BASE}/zeros1",
        "count": 100000,
        "start_index": 1,
        "precision": "9 decimal places",
        "description": "First 100,000 zeros"
    },
    "zeros2": {
        "url": f"{ODLYZKO_BASE}/zeros2",
        "count": 1000,
        "start_index": 10**12 + 1,
        "precision": "9 decimal places",
        "description": "Zeros 10^12+1 to 10^12+1000"
    },
    "zeros3": {
        "url": f"{ODLYZKO_BASE}/zeros3",
        "count": 10001,
        "start_index": 10**21 + 1,
        "precision": "12 decimal places",
        "description": "Zeros around 10^21"
    },
    "zeros4": {
        "url": f"{ODLYZKO_BASE}/zeros4",
        "count": 10001,
        "start_index": 10**22 - 10**7 + 1,
        "precision": "9 decimal places",
        "description": "Zeros around 10^22"
    },
    "zeros5": {
        "url": f"{ODLYZKO_BASE}/zeros5",
        "count": 300000,
        "start_index": 10**22 + 1,
        "precision": "8 decimal places",
        "description": "300,000 zeros starting at 10^22"
    },
    "zeros6": {
        "url": f"{ODLYZKO_BASE}/zeros6.gz",
        "count": 2001052,
        "start_index": 1,
        "precision": "8 decimal places",
        "description": "First 2,001,052 zeros (gzipped)",
        "compressed": True
    }
}

# High precision sources
HIGH_PRECISION = {
    "plouffe_100": {
        "url": "http://www.plouffe.fr/simon/constants/zeta100.html",
        "count": 100,
        "precision": "1000+ decimal places",
        "description": "First 100 zeros, ultra-high precision"
    }
}

# GIFT constants for validation
GIFT_CONSTANTS = {
    "dim_G2": 14,
    "b2": 21,
    "b3": 77,
    "H_star": 99,
    "dim_E8": 248,
    "dim_K7": 7,
    "rank_E8": 8,
    "Weyl": 5,
    "dim_J3O": 27,
    "F7": 13,  # 7th Fibonacci
    "kappa_T_inv": 61
}

print(f"Configured {len(ODLYZKO_FILES)} Odlyzko sources")
print(f"Total zeros available: {sum(f['count'] for f in ODLYZKO_FILES.values()):,}")

Configured 6 Odlyzko sources
Total zeros available: 2,422,054


## 2. Download and Cache Manager

In [4]:
class RiemannZeroCache:
    """Manages downloading and caching of Riemann zero tables."""

    def __init__(self, cache_dir: str = "./riemann_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.metadata_file = self.cache_dir / "metadata.json"
        self._load_metadata()

    def _load_metadata(self):
        if self.metadata_file.exists():
            with open(self.metadata_file) as f:
                self.metadata = json.load(f)
        else:
            self.metadata = {"downloaded": {}, "computed": {}}

    def _save_metadata(self):
        with open(self.metadata_file, 'w') as f:
            # Convert numpy types to Python types for JSON
            json.dump(self.metadata, f, indent=2, default=lambda x: float(x) if hasattr(x, 'item') else x)

    def download_odlyzko(self, name: str, force: bool = False) -> np.ndarray:
        """Download an Odlyzko table and cache locally."""
        if name not in ODLYZKO_FILES:
            raise ValueError(f"Unknown file: {name}. Available: {list(ODLYZKO_FILES.keys())}")

        info = ODLYZKO_FILES[name]
        cache_file = self.cache_dir / f"{name}.npy"

        # Check cache
        if cache_file.exists() and not force:
            print(f"Loading {name} from cache...")
            return np.load(cache_file)

        # Download
        print(f"Downloading {info['description']}...")
        response = requests.get(info['url'], stream=True)
        response.raise_for_status()

        # Parse content
        if info.get('compressed'):
            import gzip
            content = gzip.decompress(response.content).decode('utf-8')
        else:
            content = response.text

        # Parse zeros (one per line)
        zeros = []
        for line in tqdm(content.strip().split('\n'), desc="Parsing"):
            line = line.strip()
            if line and not line.startswith('#'):
                try:
                    zeros.append(float(line))
                except ValueError:
                    continue

        zeros = np.array(zeros, dtype=np.float64)

        # Cache
        np.save(cache_file, zeros)
        self.metadata['downloaded'][name] = {
            'count': len(zeros),
            'min': float(zeros.min()),
            'max': float(zeros.max()),
            'start_index': info['start_index']
        }
        self._save_metadata()

        print(f"✓ Cached {len(zeros):,} zeros to {cache_file}")
        return zeros

    def download_all(self, force: bool = False) -> dict:
        """Download all Odlyzko tables."""
        results = {}
        for name in ODLYZKO_FILES:
            try:
                results[name] = self.download_odlyzko(name, force=force)
            except Exception as e:
                print(f"⚠ Failed to download {name}: {e}")
        return results

    def get_zeros(self, start: int = 1, end: int = 100000) -> np.ndarray:
        """Get zeros by index range (1-indexed)."""
        # Load zeros1 (first 100k) or zeros6 (first 2M)
        if end <= 100000:
            zeros = self.download_odlyzko("zeros1")
        else:
            zeros = self.download_odlyzko("zeros6")

        # Convert to 0-indexed
        return zeros[start-1:end]

    def status(self):
        """Print cache status."""
        print("\n=== Riemann Zero Cache Status ===")
        total = 0
        for name, info in self.metadata.get('downloaded', {}).items():
            print(f"  {name}: {info['count']:,} zeros (γ ∈ [{info['min']:.2f}, {info['max']:.2f}])")
            total += info['count']
        print(f"  Total cached: {total:,} zeros")
        print(f"  Cache dir: {self.cache_dir.absolute()}")

# Initialize cache
cache = RiemannZeroCache()
cache.status()


=== Riemann Zero Cache Status ===
  Total cached: 0 zeros
  Cache dir: /content/riemann_cache


## 3. Download Data (Run Once)

In [5]:
# Download the main dataset (first 100,000 zeros) - quick
zeros_100k = cache.download_odlyzko("zeros1")
print(f"\nFirst 10 zeros: {zeros_100k[:10]}")
print(f"γ₁ = {zeros_100k[0]:.10f} (GIFT: dim(G₂) = 14, deviation: {abs(zeros_100k[0] - 14)/14*100:.2f}%)")

Parsing:   0%|          | 0/100000 [00:00<?, ?it/s]

✓ Cached 100,000 zeros to riemann_cache/zeros1.npy

First 10 zeros: [14.13472514 21.02203964 25.01085758 30.42487613 32.93506159 37.58617816
 40.91871901 43.32707328 48.00515088 49.77383248]
γ₁ = 14.1347251420 (GIFT: dim(G₂) = 14, deviation: 0.96%)


In [6]:
# Optional: Download full 2M dataset (takes longer, ~40MB)
# Uncomment to run:
# zeros_2M = cache.download_odlyzko("zeros6")
# print(f"Loaded {len(zeros_2M):,} zeros")

## 4. GPU-Accelerated Analysis with CuPy

In [7]:
class GIFTRiemannAnalyzer:
    """GPU-accelerated analysis of GIFT-Riemann correspondences."""

    def __init__(self, zeros: np.ndarray, use_gpu: bool = True):
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

        # Transfer to GPU if available
        if self.use_gpu:
            self.zeros = cp.asarray(zeros)
            print(f"✓ Loaded {len(zeros):,} zeros on GPU")
        else:
            self.zeros = zeros
            print(f"✓ Loaded {len(zeros):,} zeros on CPU")

    def find_near_integer(self, target: int, tolerance: float = 0.5) -> dict:
        """Find zeros nearest to a target integer."""
        xp = self.xp
        diff = xp.abs(self.zeros - target)
        mask = diff < tolerance

        if not xp.any(mask):
            # Find closest anyway
            idx = int(xp.argmin(diff))
            return {
                'target': target,
                'nearest_index': idx + 1,  # 1-indexed
                'value': float(self.zeros[idx]),
                'deviation': float(diff[idx]),
                'deviation_pct': float(diff[idx] / target * 100)
            }

        indices = xp.where(mask)[0]
        if self.use_gpu:
            indices = indices.get()
            values = self.zeros[mask].get()
            deviations = diff[mask].get()
        else:
            values = self.zeros[mask]
            deviations = diff[mask]

        best_idx = int(indices[np.argmin(deviations)])
        return {
            'target': target,
            'matches': len(indices),
            'nearest_index': best_idx + 1,
            'value': float(self.zeros[best_idx]),
            'deviation': float(np.min(deviations)),
            'deviation_pct': float(np.min(deviations) / target * 100)
        }

    def validate_gift_correspondences(self) -> dict:
        """Validate all GIFT-Riemann correspondences."""
        results = {}

        # Known correspondences from research
        correspondences = [
            (1, 14, "dim(G₂)"),
            (2, 21, "b₂"),
            (20, 77, "b₃"),
            (29, 99, "H*"),
            (107, 248, "dim(E₈)"),
        ]

        print("\n=== GIFT-Riemann Correspondences ===")
        print(f"{'Index':<8} {'γₙ':<12} {'Target':<8} {'Constant':<12} {'Dev %':<10}")
        print("-" * 55)

        for idx, target, name in correspondences:
            if idx <= len(self.zeros):
                gamma = float(self.zeros[idx - 1])  # 0-indexed
                dev_pct = abs(gamma - target) / target * 100
                results[name] = {
                    'index': idx,
                    'gamma': gamma,
                    'target': target,
                    'deviation_pct': dev_pct
                }
                print(f"γ_{idx:<5} {gamma:<12.6f} {target:<8} {name:<12} {dev_pct:<10.4f}")

        mean_dev = np.mean([r['deviation_pct'] for r in results.values()])
        print(f"\nMean deviation: {mean_dev:.4f}%")
        results['mean_deviation'] = mean_dev

        return results

    def test_pell_equation(self) -> dict:
        """Test the modified Pell equation: γ₂₉² - 49γ₁² + γ₂ + 1 ≈ 0"""
        g1 = float(self.zeros[0])   # γ₁
        g2 = float(self.zeros[1])   # γ₂
        g29 = float(self.zeros[28]) # γ₂₉

        # Modified Pell: γ₂₉² - 49γ₁² + γ₂ + 1 ≈ 0
        pell_value = g29**2 - 49 * g1**2 + g2 + 1

        print("\n=== Modified Pell Equation ===")
        print(f"γ₁  = {g1:.10f}")
        print(f"γ₂  = {g2:.10f}")
        print(f"γ₂₉ = {g29:.10f}")
        print(f"\nγ₂₉² - 49γ₁² + γ₂ + 1 = {pell_value:.10f}")
        print(f"Expected: 0")
        print(f"Relative error: {abs(pell_value) / g29**2 * 100:.6f}%")

        return {
            'gamma_1': g1,
            'gamma_2': g2,
            'gamma_29': g29,
            'pell_value': pell_value,
            'relative_error_pct': abs(pell_value) / g29**2 * 100
        }

    def test_recurrence(self, n_test: int = 1000) -> dict:
        """Test the GIFT recurrence: γₙ ≈ a₅γₙ₋₅ + a₈γₙ₋₈ + a₁₃γₙ₋₁₃ + a₂₇γₙ₋₂₇ + c"""
        xp = self.xp

        # Need at least 28 zeros for lag-27
        if len(self.zeros) < 28 + n_test:
            n_test = len(self.zeros) - 28

        # GIFT lags: Weyl=5, rank(E₈)=8, F₇=13, dim(J₃(O))=27
        lags = [5, 8, 13, 27]

        # Build design matrix for linear regression
        # γₙ = a₅γₙ₋₅ + a₈γₙ₋₈ + a₁₃γₙ₋₁₃ + a₂₇γₙ₋₂₇ + c
        start = max(lags)
        end = start + n_test

        if self.use_gpu:
            zeros_cpu = self.zeros.get()
        else:
            zeros_cpu = self.zeros

        # Design matrix
        X = np.column_stack([
            zeros_cpu[start - lag:end - lag] for lag in lags
        ] + [np.ones(n_test)])  # constant term

        y = zeros_cpu[start:end]

        # Least squares fit
        coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

        # Predictions and errors
        y_pred = X @ coeffs
        errors = np.abs(y - y_pred)
        rel_errors = errors / y * 100

        print(f"\n=== Recurrence Analysis (n={n_test:,}) ===")
        print(f"\nFitted coefficients:")
        for lag, coef in zip(lags, coeffs[:-1]):
            print(f"  a_{lag} = {coef:.6f}")
        print(f"  c   = {coeffs[-1]:.6f}")
        print(f"\nError statistics:")
        print(f"  Mean relative error: {np.mean(rel_errors):.4f}%")
        print(f"  Max relative error:  {np.max(rel_errors):.4f}%")
        print(f"  Std relative error:  {np.std(rel_errors):.4f}%")

        return {
            'lags': lags,
            'coefficients': {f'a_{lag}': float(c) for lag, c in zip(lags, coeffs[:-1])},
            'constant': float(coeffs[-1]),
            'mean_rel_error_pct': float(np.mean(rel_errors)),
            'max_rel_error_pct': float(np.max(rel_errors)),
            'std_rel_error_pct': float(np.std(rel_errors)),
            'n_samples': n_test
        }

    def scan_for_gift_constants(self, tolerance_pct: float = 1.0) -> list:
        """Scan all zeros for proximity to GIFT constants."""
        results = []

        for name, value in GIFT_CONSTANTS.items():
            match = self.find_near_integer(value, tolerance=value * tolerance_pct / 100)
            match['gift_constant'] = name
            results.append(match)

        print(f"\n=== Scan for GIFT Constants (tolerance: {tolerance_pct}%) ===")
        print(f"{'Constant':<15} {'Value':<8} {'γₙ Index':<10} {'γₙ Value':<12} {'Dev %':<8}")
        print("-" * 60)

        for r in sorted(results, key=lambda x: x['deviation_pct']):
            print(f"{r['gift_constant']:<15} {r['target']:<8} γ_{r['nearest_index']:<7} {r['value']:<12.6f} {r['deviation_pct']:<8.4f}")

        return results

# Initialize analyzer
analyzer = GIFTRiemannAnalyzer(zeros_100k)

✓ Loaded 100,000 zeros on GPU


## 5. Run GIFT-Riemann Validation

In [8]:
# Validate known correspondences
correspondences = analyzer.validate_gift_correspondences()


=== GIFT-Riemann Correspondences ===
Index    γₙ           Target   Constant     Dev %     
-------------------------------------------------------
γ_1     14.134725    14       dim(G₂)      0.9623    
γ_2     21.022040    21       b₂           0.1050    
γ_20    77.144840    77       b₃           0.1881    
γ_29    98.831194    99       H*           0.1705    
γ_107   248.101990   248      dim(E₈)      0.0411    

Mean deviation: 0.2934%


In [9]:
# Test modified Pell equation
pell = analyzer.test_pell_equation()


=== Modified Pell Equation ===
γ₁  = 14.1347251420
γ₂  = 21.0220396390
γ₂₉ = 98.8311942180

γ₂₉² - 49γ₁² + γ₂ + 1 = -0.1052969594
Expected: 0
Relative error: 0.001078%


In [10]:
# Test recurrence relation
recurrence = analyzer.test_recurrence(n_test=10000)


=== Recurrence Analysis (n=10,000) ===

Fitted coefficients:
  a_5 = 0.510445
  a_8 = 0.668152
  a_13 = 0.132119
  a_27 = -0.310764
  c   = 1.443065

Error statistics:
  Mean relative error: 0.0147%
  Max relative error:  4.7770%
  Std relative error:  0.0791%


In [11]:
# Scan for all GIFT constants
scan = analyzer.scan_for_gift_constants(tolerance_pct=2.0)


=== Scan for GIFT Constants (tolerance: 2.0%) ===
Constant        Value    γₙ Index   γₙ Value     Dev %   
------------------------------------------------------------
dim_E8          248      γ_107     248.101990   0.0411  
b2              21       γ_2       21.022040    0.1050  
H_star          99       γ_29      98.831194    0.1705  
b3              77       γ_20      77.144840    0.1881  
kappa_T_inv     61       γ_14      60.831779    0.2758  
dim_G2          14       γ_1       14.134725    0.9623  
dim_J3O         27       γ_3       25.010858    7.3672  
F7              13       γ_1       14.134725    8.7287  
rank_E8         8        γ_1       14.134725    76.6841 
dim_K7          7        γ_1       14.134725    101.9246
Weyl            5        γ_1       14.134725    182.6945


## 6. High-Precision Computation with mpmath

In [12]:
def compute_zeros_mpmath(start: int, count: int, precision: int = 50) -> list:
    """Compute zeros using mpmath (high precision but slow)."""
    if not MPMATH_AVAILABLE:
        raise ImportError("mpmath not available")

    mp.dps = precision
    zeros = []

    for n in tqdm(range(start, start + count), desc=f"Computing zeros (precision={precision})"):
        z = zetazero(n)
        zeros.append(float(z.imag))

    return zeros

# Example: compute first 10 zeros with 100 digits precision
if MPMATH_AVAILABLE:
    high_prec_zeros = compute_zeros_mpmath(1, 10, precision=100)
    print("\nHigh-precision zeros (100 digits):")
    for i, z in enumerate(high_prec_zeros, 1):
        print(f"  γ_{i} = {z}")

Computing zeros (precision=100):   0%|          | 0/10 [00:00<?, ?it/s]


High-precision zeros (100 digits):
  γ_1 = 14.134725141734695
  γ_2 = 21.022039638771556
  γ_3 = 25.01085758014569
  γ_4 = 30.424876125859512
  γ_5 = 32.93506158773919
  γ_6 = 37.586178158825675
  γ_7 = 40.9187190121475
  γ_8 = 43.327073280915
  γ_9 = 48.00515088116716
  γ_10 = 49.7738324776723


## 7. GPU-Accelerated Spectral Analysis

In [13]:
def spectral_analysis_gpu(zeros: np.ndarray, max_freq: int = 1000):
    """GPU-accelerated FFT analysis of zero spacings."""
    if not GPU_AVAILABLE:
        print("GPU not available, using CPU")
        xp = np
    else:
        xp = cp
        zeros = cp.asarray(zeros)

    # Compute spacings
    spacings = xp.diff(zeros)

    # Normalize spacings
    mean_spacing = xp.mean(spacings)
    normalized = spacings / mean_spacing

    # FFT
    fft = xp.fft.fft(normalized)
    power = xp.abs(fft[:max_freq])**2
    freqs = xp.fft.fftfreq(len(normalized))[:max_freq]

    if GPU_AVAILABLE:
        power = power.get()
        freqs = freqs.get()
        cp.get_default_memory_pool().free_all_blocks()

    return freqs, power

# Run spectral analysis
freqs, power = spectral_analysis_gpu(zeros_100k)

# Find dominant frequencies
top_indices = np.argsort(power)[-10:][::-1]
print("\n=== Top 10 Spectral Peaks ===")
for i, idx in enumerate(top_indices):
    print(f"  {i+1}. freq={freqs[idx]:.6f}, power={power[idx]:.2f}")


=== Top 10 Spectral Peaks ===
  1. freq=0.000000, power=9999800001.00
  2. freq=0.000010, power=27769030.78
  3. freq=0.000020, power=12113324.26
  4. freq=0.000030, power=7285162.19
  5. freq=0.000040, power=5039575.54
  6. freq=0.000050, power=3772944.78
  7. freq=0.000060, power=2972313.08
  8. freq=0.000070, power=2426492.60
  9. freq=0.000080, power=2033746.14
  10. freq=0.000090, power=1739453.67


## 8. Save Results

In [14]:
def save_analysis_results(filename: str = "gift_riemann_analysis.json"):
    """Save all analysis results to JSON."""
    results = {
        'correspondences': correspondences,
        'pell_equation': pell,
        'recurrence': recurrence,
        'gift_constants_scan': [{k: (float(v) if isinstance(v, (np.floating, float)) else v)
                                  for k, v in s.items()} for s in scan],
        'metadata': {
            'n_zeros': len(zeros_100k),
            'gpu_used': GPU_AVAILABLE,
            'source': 'Odlyzko zeros1'
        }
    }

    with open(filename, 'w') as f:
        json.dump(results, f, indent=2, default=lambda x: float(x) if hasattr(x, 'item') else str(x))

    print(f"✓ Results saved to {filename}")
    return results

results = save_analysis_results()

✓ Results saved to gift_riemann_analysis.json


## 9. Quick API Reference

```python
# Initialize cache and download data
cache = RiemannZeroCache()
zeros = cache.download_odlyzko("zeros1")  # First 100k
zeros = cache.download_odlyzko("zeros6")  # First 2M
zeros = cache.get_zeros(1, 1000)          # Get range by index

# Analyze with GPU
analyzer = GIFTRiemannAnalyzer(zeros, use_gpu=True)
analyzer.validate_gift_correspondences()
analyzer.test_pell_equation()
analyzer.test_recurrence(n_test=10000)
analyzer.scan_for_gift_constants(tolerance_pct=1.0)
analyzer.find_near_integer(target=14)     # Find zeros near any integer

# High-precision computation
zeros_hp = compute_zeros_mpmath(1, 100, precision=100)
```

In [15]:
# Final status
cache.status()
print("\n✓ Notebook ready for GIFT-Riemann research!")


=== Riemann Zero Cache Status ===
  zeros1: 100,000 zeros (γ ∈ [14.13, 74920.83])
  Total cached: 100,000 zeros
  Cache dir: /content/riemann_cache

✓ Notebook ready for GIFT-Riemann research!


In [16]:
# === CELL 1: Indices "doublement GIFT" ===
# Chercher les indices n où n ET γₙ sont des constantes GIFT

gift_values = [7, 8, 13, 14, 21, 27, 61, 77, 99, 248]
gift_indices = [1, 2, 3, 5, 7, 8, 13, 14, 20, 21, 27, 29, 77, 99, 107]

print("=== Indices Doublement GIFT ===")
print(f"{'Index n':<10} {'γₙ':<15} {'Nearest GIFT':<15} {'Dev %':<10}")
print("-" * 55)

for n in gift_indices:
    if n <= len(zeros_100k):
        gamma = zeros_100k[n-1]
        # Trouver la constante GIFT la plus proche
        nearest = min(gift_values, key=lambda x: abs(gamma - x))
        dev = abs(gamma - nearest) / nearest * 100
        marker = "⭐" if dev < 1.0 else ""
        print(f"{n:<10} {gamma:<15.6f} {nearest:<15} {dev:<10.4f} {marker}")


=== Indices Doublement GIFT ===
Index n    γₙ              Nearest GIFT    Dev %     
-------------------------------------------------------
1          14.134725       14              0.9623     ⭐
2          21.022040       21              0.1050     ⭐
3          25.010858       27              7.3672     
5          32.935062       27              21.9817    
7          40.918719       27              51.5508    
8          43.327073       27              60.4706    
13         59.347044       61              2.7098     
14         60.831779       61              0.2758     ⭐
20         77.144840       77              0.1881     ⭐
21         79.337375       77              3.0356     
27         94.651344       99              4.3926     
29         98.831194       99              0.1705     ⭐
77         195.265397      248             21.2640    
99         233.693404      248             5.7688     
107        248.101990      248             0.0411     ⭐


In [17]:
# === CELL 2: Test de Pell généralisé ===
# Tester d'autres triplets (i, j, k) pour γₖ² - c·γᵢ² + γⱼ + d ≈ 0

import itertools

def test_pell_form(zeros, i, j, k, max_c=100):
    """Cherche c, d tels que γₖ² - c·γᵢ² + γⱼ + d ≈ 0"""
    gi, gj, gk = zeros[i-1], zeros[j-1], zeros[k-1]

    best_c, best_d, best_err = 0, 0, float('inf')
    for c in range(1, max_c):
        # d = -(γₖ² - c·γᵢ² + γⱼ)
        d_approx = -(gk**2 - c * gi**2 + gj)
        d = round(d_approx)
        err = abs(gk**2 - c * gi**2 + gj + d)
        if err < best_err:
            best_c, best_d, best_err = c, d, err

    return best_c, best_d, best_err

# Tester des triplets intéressants
triplets = [
    (1, 2, 29),   # Original: γ₂₉² - 49γ₁² + γ₂ + 1 ≈ 0
    (1, 2, 20),   # Avec b₃
    (1, 2, 107),  # Avec dim(E₈)
    (2, 1, 29),   # Inversé
    (1, 14, 29),  # Avec γ₁₄ ≈ 61
    (1, 20, 107), # b₃ vers E₈
]

print("=== Pell Généralisé: γₖ² - c·γᵢ² + γⱼ + d ≈ 0 ===")
print(f"{'(i,j,k)':<15} {'c':<6} {'d':<6} {'Erreur':<12} {'c interp.':<20}")
print("-" * 70)

for i, j, k in triplets:
    c, d, err = test_pell_form(zeros_100k, i, j, k)
    # Interpréter c
    interp = ""
    if c == 49: interp = "7² = dim(K₇)²"
    elif c == 14: interp = "dim(G₂)"
    elif c == 21: interp = "b₂"
    elif c == 77: interp = "b₃"
    elif c == 99: interp = "H*"

    print(f"({i},{j},{k})".ljust(15) + f"{c:<6} {d:<6} {err:<12.6f} {interp:<20}")


=== Pell Généralisé: γₖ² - c·γᵢ² + γⱼ + d ≈ 0 ===
(i,j,k)         c      d      Erreur       c interp.           
----------------------------------------------------------------------
(1,2,29)       40     -1797  0.008797                         
(1,2,20)       89     11809  0.002092                         
(1,2,107)      40     -53584 0.001318                         
(2,1,29)       17     -2269  0.004884                         
(1,14,29)      17     -6432  0.001003                         
(1,20,107)     6      -60433 0.000417                         


In [18]:
# === CELL 3: Spacings entre zéros GIFT ===
# Analyser les espacements γₙ₊₁ - γₙ aux indices GIFT

gift_indices_extended = [1, 2, 3, 5, 7, 8, 13, 14, 20, 21, 27, 29, 42, 61, 77, 99]

print("=== Espacements aux Indices GIFT ===")
print(f"{'Index n':<8} {'γₙ':<12} {'γₙ₊₁ - γₙ':<12} {'Ratio to mean':<15}")
print("-" * 50)

mean_spacing = np.mean(np.diff(zeros_100k[:1000]))

for n in gift_indices_extended:
    if n < len(zeros_100k):
        gamma_n = zeros_100k[n-1]
        spacing = zeros_100k[n] - zeros_100k[n-1]
        ratio = spacing / mean_spacing
        print(f"{n:<8} {gamma_n:<12.4f} {spacing:<12.6f} {ratio:<15.4f}")

print(f"\nMean spacing (first 1000): {mean_spacing:.6f}")


=== Espacements aux Indices GIFT ===
Index n  γₙ           γₙ₊₁ - γₙ    Ratio to mean  
--------------------------------------------------
1        14.1347      6.887314     4.8961         
2        21.0220      3.988818     2.8356         
3        25.0109      5.414019     3.8488         
5        32.9351      4.651117     3.3064         
7        40.9187      2.408354     1.7121         
8        43.3271      4.678078     3.3256         
13       59.3470      1.484735     1.0555         
14       60.8318      4.280766     3.0431         
20       77.1448      2.192535     1.5586         
21       79.3374      3.573006     2.5400         
27       94.6513      1.219290     0.8668         
29       98.8312      2.486657     1.7677         
42       127.5167     2.062020     1.4659         
61       165.5371     1.647371     1.1711         
77       195.2654     1.611085     1.1453         
99       233.6934     2.830825     2.0124         

Mean spacing (first 1000): 1.406694


In [19]:
# === CELL 4: Récurrence avec coefficients GIFT ===
# Tester si les coefficients de la récurrence sont eux-mêmes des ratios GIFT

# Reprendre les coefficients trouvés
a5, a8, a13, a27 = 0.5104, 0.6681, 0.1321, -0.3108
c_const = 1.443

print("=== Interprétation des Coefficients ===\n")

# Chercher des ratios GIFT proches
gift_ratios = {
    "1/2": 0.5,
    "2/3 (Koide)": 2/3,
    "7/14": 7/14,
    "14/21": 14/21,
    "21/77": 21/77,
    "14/99": 14/99,
    "7/99": 7/99,
    "1/7": 1/7,
    "1/3": 1/3,
}

for name, val in [("a_5", a5), ("a_8", a8), ("a_13", a13), ("a_27", a27)]:
    print(f"{name} = {val:.4f}")
    for ratio_name, ratio_val in gift_ratios.items():
        if abs(val - ratio_val) < 0.02 or abs(val + ratio_val) < 0.02:
            print(f"  → proche de {ratio_name} = {ratio_val:.4f} (diff: {abs(val) - ratio_val:.4f})")
    print()


=== Interprétation des Coefficients ===

a_5 = 0.5104
  → proche de 1/2 = 0.5000 (diff: 0.0104)
  → proche de 7/14 = 0.5000 (diff: 0.0104)

a_8 = 0.6681
  → proche de 2/3 (Koide) = 0.6667 (diff: 0.0014)
  → proche de 14/21 = 0.6667 (diff: 0.0014)

a_13 = 0.1321
  → proche de 14/99 = 0.1414 (diff: -0.0093)
  → proche de 1/7 = 0.1429 (diff: -0.0108)

a_27 = -0.3108



In [20]:
# === CELL 5: Sommes et produits de zéros GIFT ===
# Tester des relations algébriques

g = zeros_100k  # alias

print("=== Relations Algébriques ===\n")

# Sommes
print("Sommes:")
print(f"γ₁ + γ₂ = {g[0] + g[1]:.6f}  (35.157, proche de 35 = 5×7?)")
print(f"γ₁ + γ₂₀ = {g[0] + g[19]:.6f}  (91.28, proche de 91 = 7×13?)")
print(f"γ₂ + γ₂₀ = {g[1] + g[19]:.6f}  (98.17, proche de H* = 99?)")

print("\nProduits:")
print(f"γ₁ × γ₂ / 10 = {g[0] * g[1] / 10:.6f}  (proche de 30 = h_E8?)")
print(f"γ₁ × γ₂₀ / 100 = {g[0] * g[19] / 100:.6f}  (proche de 11 = D_bulk?)")

print("\nRatios:")
print(f"γ₂₀ / γ₂ = {g[19] / g[1]:.6f}  (proche de 77/21 = 11/3?)")
print(f"γ₂₉ / γ₁ = {g[28] / g[0]:.6f}  (proche de 99/14 = H*/dim(G₂)?)")
print(f"γ₁₀₇ / γ₂₉ = {g[106] / g[28]:.6f}  (proche de 248/99?)")

print("\nRatios GIFT exacts vs observés:")
print(f"77/21 = {77/21:.6f} vs γ₂₀/γ₂ = {g[19]/g[1]:.6f} → diff: {abs(77/21 - g[19]/g[1]):.6f}")
print(f"99/14 = {99/14:.6f} vs γ₂₉/γ₁ = {g[28]/g[0]:.6f} → diff: {abs(99/14 - g[28]/g[0]):.6f}")
print(f"248/99 = {248/99:.6f} vs γ₁₀₇/γ₂₉ = {g[106]/g[28]:.6f} → diff: {abs(248/99 - g[106]/g[28]):.6f}")


=== Relations Algébriques ===

Sommes:
γ₁ + γ₂ = 35.156765  (35.157, proche de 35 = 5×7?)
γ₁ + γ₂₀ = 91.279565  (91.28, proche de 91 = 7×13?)
γ₂ + γ₂₀ = 98.166880  (98.17, proche de H* = 99?)

Produits:
γ₁ × γ₂ / 10 = 29.714075  (proche de 30 = h_E8?)
γ₁ × γ₂₀ / 100 = 10.904211  (proche de 11 = D_bulk?)

Ratios:
γ₂₀ / γ₂ = 3.669712  (proche de 77/21 = 11/3?)
γ₂₉ / γ₁ = 6.992085  (proche de 99/14 = H*/dim(G₂)?)
γ₁₀₇ / γ₂₉ = 2.510361  (proche de 248/99?)

Ratios GIFT exacts vs observés:
77/21 = 3.666667 vs γ₂₀/γ₂ = 3.669712 → diff: 0.003046
99/14 = 7.071429 vs γ₂₉/γ₁ = 6.992085 → diff: 0.079344
248/99 = 2.505051 vs γ₁₀₇/γ₂₉ = 2.510361 → diff: 0.005311


In [21]:
# === CELL 6: Chercher TOUS les zéros proches d'entiers GIFT ===
# Scan exhaustif

all_gift = [2, 3, 5, 7, 8, 11, 13, 14, 21, 27, 32, 42, 49, 52, 61, 65, 77, 78, 91, 99, 120, 128, 144, 163, 196, 240, 248, 496]

print("=== Scan Exhaustif: Zéros < 1% d'un entier GIFT ===\n")
print(f"{'GIFT val':<10} {'γₙ index':<12} {'γₙ value':<14} {'Dev %':<10}")
print("-" * 50)

found = []
for target in all_gift:
    # Chercher le zéro le plus proche
    diffs = np.abs(zeros_100k - target)
    idx = np.argmin(diffs)
    dev_pct = diffs[idx] / target * 100

    if dev_pct < 1.0:  # Moins de 1%
        found.append((target, idx+1, zeros_100k[idx], dev_pct))
        print(f"{target:<10} γ_{idx+1:<10} {zeros_100k[idx]:<14.6f} {dev_pct:<10.4f}")

print(f"\nTrouvé {len(found)} correspondances < 1%")


=== Scan Exhaustif: Zéros < 1% d'un entier GIFT ===

GIFT val   γₙ index     γₙ value       Dev %     
--------------------------------------------------
14         γ_1          14.134725      0.9623    
21         γ_2          21.022040      0.1050    
61         γ_14         60.831779      0.2758    
65         γ_15         65.112544      0.1731    
77         γ_20         77.144840      0.1881    
99         γ_29         98.831194      0.1705    
128        γ_42         127.516684     0.3776    
144        γ_50         143.111846     0.6168    
163        γ_60         163.030710     0.0188    
196        γ_77         195.265397     0.3748    
240        γ_102        239.555478     0.1852    
248        γ_107        248.101990     0.0411    
496        γ_268        496.429696     0.0866    

Trouvé 13 correspondances < 1%


In [22]:
# === CELL 7: Le cas γ₆₀ ≈ 163 — Heegner magic ===
# 60 = 61-1 = κ_T⁻¹ - 1, et 163 est le plus grand nombre de Heegner !

heegner = [1, 2, 3, 7, 11, 19, 43, 67, 163]

print("=== Correspondances Heegner ===\n")
print(f"{'Heegner':<10} {'γₙ index':<12} {'γₙ value':<14} {'Dev %':<10} {'Index note':<20}")
print("-" * 70)

for h in heegner:
    diffs = np.abs(zeros_100k - h)
    idx = np.argmin(diffs)
    dev_pct = diffs[idx] / h * 100

    # Note sur l'indice
    idx_note = ""
    if idx + 1 in [1, 2, 3, 5, 7, 8, 13, 14, 21, 27, 29, 61, 77, 99]:
        idx_note = "GIFT index!"

    marker = "⭐" if dev_pct < 1.0 else ""
    print(f"{h:<10} γ_{idx+1:<10} {zeros_100k[idx]:<14.6f} {dev_pct:<10.4f} {idx_note} {marker}")

print(f"\nγ₆₀ = {zeros_100k[59]:.10f}")
print(f"163 exactement")
print(f"Différence: {zeros_100k[59] - 163:.10f}")
print(f"Note: 60 = 61 - 1 = κ_T⁻¹ - 1")


=== Correspondances Heegner ===

Heegner    γₙ index     γₙ value       Dev %      Index note          
----------------------------------------------------------------------
1          γ_1          14.134725      1313.4725  GIFT index! 
2          γ_1          14.134725      606.7363   GIFT index! 
3          γ_1          14.134725      371.1575   GIFT index! 
7          γ_1          14.134725      101.9246   GIFT index! 
11         γ_1          14.134725      28.4975    GIFT index! 
19         γ_2          21.022040      10.6423    GIFT index! 
43         γ_8          43.327073      0.7606     GIFT index! ⭐
67         γ_16         67.079811      0.1191      ⭐
163        γ_60         163.030710     0.0188      ⭐

γ₆₀ = 163.0307096870
163 exactement
Différence: 0.0307096870
Note: 60 = 61 - 1 = κ_T⁻¹ - 1


In [23]:
# === CELL 8: Pattern dans les indices ===
# Les indices des correspondances forment-ils une structure ?

matches = [
    (1, 14), (2, 21), (14, 61), (15, 65), (20, 77),
    (29, 99), (42, 128), (50, 144), (60, 163),
    (77, 196), (102, 240), (107, 248), (268, 496)
]

print("=== Analyse des Indices ===\n")

indices = [m[0] for m in matches]
values = [m[1] for m in matches]

print("Différences entre indices consécutifs:")
idx_diffs = np.diff(indices)
print(idx_diffs)
print(f"\nIndices: {indices}")
print(f"\nFactorisations des indices:")
for idx in indices:
    factors = []
    n = idx
    for p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]:
        while n % p == 0:
            factors.append(p)
            n //= p
    if n > 1:
        factors.append(n)
    gift_rel = ""
    if idx == 14: gift_rel = "= dim(G₂)"
    if idx == 77: gift_rel = "= b₃"
    if idx == 42: gift_rel = "= 2×21 = 2×b₂"
    if idx == 29: gift_rel = "≈ H*/3.4"
    if idx == 60: gift_rel = "= κ_T⁻¹ - 1"
    print(f"  {idx:>3} = {' × '.join(map(str, factors)):<15} {gift_rel}")


=== Analyse des Indices ===

Différences entre indices consécutifs:
[  1  12   1   5   9  13   8  10  17  25   5 161]

Indices: [1, 2, 14, 15, 20, 29, 42, 50, 60, 77, 102, 107, 268]

Factorisations des indices:
    1 =                 
    2 = 2               
   14 = 2 × 7           = dim(G₂)
   15 = 3 × 5           
   20 = 2 × 2 × 5       
   29 = 29              ≈ H*/3.4
   42 = 2 × 3 × 7       = 2×21 = 2×b₂
   50 = 2 × 5 × 5       
   60 = 2 × 2 × 3 × 5   = κ_T⁻¹ - 1
   77 = 7 × 11          = b₃
  102 = 2 × 3 × 17      
  107 = 107             
  268 = 2 × 2 × 67      


In [24]:
# === CELL 9: Triple zéros et classes de torsion ===
# Les 4 classes de torsion ont dim 1, 7, 14, 27 → total 49
# Tester des sommes de 3 zéros

print("=== Sommes de Triplets de Zéros ===\n")

# Triplets intéressants
triplets = [
    (1, 7, 14),   # Fibres: 1, dim(K₇), dim(G₂)
    (1, 14, 27),  # Classes torsion: W₁, W₁₄, W₂₇
    (2, 20, 29),  # Betti: b₂-idx, b₃-idx, H*-idx
    (1, 2, 20),   # Premiers correspondants
]

for t in triplets:
    i, j, k = t
    s = zeros_100k[i-1] + zeros_100k[j-1] + zeros_100k[k-1]
    print(f"γ_{i} + γ_{j} + γ_{k} = {s:.4f}")

    # Chercher constante GIFT proche
    gift_targets = [49, 77, 91, 99, 119, 126, 133, 147, 168, 196, 248]
    nearest = min(gift_targets, key=lambda x: abs(s - x))
    dev = abs(s - nearest) / nearest * 100
    print(f"  → Proche de {nearest} (dev: {dev:.2f}%)\n")

# Somme des 4 premiers (liée aux 4 classes de torsion?)
s4 = sum(zeros_100k[:4])
print(f"γ₁ + γ₂ + γ₃ + γ₄ = {s4:.4f}")
print(f"  → 4 classes de torsion: dim = 1+7+14+27 = 49")
print(f"  → Ratio: {s4/49:.4f}")


=== Sommes de Triplets de Zéros ===

γ_1 + γ_7 + γ_14 = 115.8852
  → Proche de 119 (dev: 2.62%)

γ_1 + γ_14 + γ_27 = 169.6178
  → Proche de 168 (dev: 0.96%)

γ_2 + γ_20 + γ_29 = 196.9981
  → Proche de 196 (dev: 0.51%)

γ_1 + γ_2 + γ_20 = 112.3016
  → Proche de 119 (dev: 5.63%)

γ₁ + γ₂ + γ₃ + γ₄ = 90.5925
  → 4 classes de torsion: dim = 1+7+14+27 = 49
  → Ratio: 1.8488


In [25]:
# === CELL 10: Produits normalisés ===
# Explorer γᵢ × γⱼ / constante

print("=== Produits Normalisés ===\n")

products = [
    (1, 2, "γ₁ × γ₂"),
    (1, 20, "γ₁ × γ₂₀"),
    (2, 20, "γ₂ × γ₂₀"),
    (1, 29, "γ₁ × γ₂₉"),
    (14, 29, "γ₁₄ × γ₂₉"),
]

normalizers = [7, 14, 21, 77, 99, 100, 10]

for i, j, name in products:
    prod = zeros_100k[i-1] * zeros_100k[j-1]
    print(f"{name} = {prod:.4f}")
    for norm in normalizers:
        result = prod / norm
        # Chercher si proche d'un entier GIFT
        nearest_int = round(result)
        if nearest_int in [7, 8, 11, 13, 14, 21, 27, 30, 42, 61, 77, 99, 120, 128, 248]:
            dev = abs(result - nearest_int) / nearest_int * 100
            if dev < 5:
                print(f"  ÷ {norm} = {result:.4f} ≈ {nearest_int} (dev: {dev:.2f}%)")
    print()


=== Produits Normalisés ===

γ₁ × γ₂ = 297.1408
  ÷ 7 = 42.4487 ≈ 42 (dev: 1.07%)
  ÷ 14 = 21.2243 ≈ 21 (dev: 1.07%)
  ÷ 21 = 14.1496 ≈ 14 (dev: 1.07%)
  ÷ 10 = 29.7141 ≈ 30 (dev: 0.95%)

γ₁ × γ₂₀ = 1090.4211
  ÷ 77 = 14.1613 ≈ 14 (dev: 1.15%)
  ÷ 99 = 11.0144 ≈ 11 (dev: 0.13%)
  ÷ 100 = 10.9042 ≈ 11 (dev: 0.87%)

γ₂ × γ₂₀ = 1621.7419
  ÷ 21 = 77.2258 ≈ 77 (dev: 0.29%)
  ÷ 77 = 21.0616 ≈ 21 (dev: 0.29%)

γ₁ × γ₂₉ = 1396.9518
  ÷ 99 = 14.1106 ≈ 14 (dev: 0.79%)
  ÷ 100 = 13.9695 ≈ 14 (dev: 0.22%)

γ₁₄ × γ₂₉ = 6012.0773
  ÷ 99 = 60.7281 ≈ 61 (dev: 0.45%)



In [26]:
# === CELL 11: La séquence des correspondances ===
# Chercher une formule pour prédire l'indice n où γₙ ≈ GIFT_constant

import scipy.stats as stats

matches_sorted = sorted([
    (14, 1), (21, 2), (61, 14), (65, 15), (77, 20),
    (99, 29), (128, 42), (144, 50), (163, 60),
    (196, 77), (240, 102), (248, 107), (496, 268)
], key=lambda x: x[0])

gift_vals = np.array([m[0] for m in matches_sorted])
indices = np.array([m[1] for m in matches_sorted])

print("=== Régression: Index vs Valeur GIFT ===\n")

# Régression linéaire
slope, intercept, r, p, se = stats.linregress(gift_vals, indices)
print(f"Régression linéaire: n ≈ {slope:.4f} × GIFT + {intercept:.4f}")
print(f"R² = {r**2:.4f}")

# Prédictions
print(f"\nPrédictions:")
for gv in [14, 21, 61, 77, 99, 163, 248, 496]:
    pred = slope * gv + intercept
    actual = dict(matches_sorted).get(gv, "?")
    print(f"  GIFT={gv}: prédit n={pred:.1f}, réel n={actual}")

# Ratio index/valeur
print(f"\nRatio index/valeur:")
for gv, idx in matches_sorted:
    ratio = idx / gv
    print(f"  {idx}/{gv} = {ratio:.4f}")


=== Régression: Index vs Valeur GIFT ===

Régression linéaire: n ≈ 0.5519 × GIFT + -22.3308
R² = 0.9829

Prédictions:
  GIFT=14: prédit n=-14.6, réel n=1
  GIFT=21: prédit n=-10.7, réel n=2
  GIFT=61: prédit n=11.3, réel n=14
  GIFT=77: prédit n=20.2, réel n=20
  GIFT=99: prédit n=32.3, réel n=29
  GIFT=163: prédit n=67.6, réel n=60
  GIFT=248: prédit n=114.5, réel n=107
  GIFT=496: prédit n=251.4, réel n=268

Ratio index/valeur:
  1/14 = 0.0714
  2/21 = 0.0952
  14/61 = 0.2295
  15/65 = 0.2308
  20/77 = 0.2597
  29/99 = 0.2929
  42/128 = 0.3281
  50/144 = 0.3472
  60/163 = 0.3681
  77/196 = 0.3929
  102/240 = 0.4250
  107/248 = 0.4315
  268/496 = 0.5403


In [27]:
# === CELL 12: Vérification γ₂ × γ₂₀ = b₂ × b₃ ===

print("=== Relation Multiplicative Fondamentale ===\n")

g2, g20 = zeros_100k[1], zeros_100k[19]
b2, b3 = 21, 77

prod_gamma = g2 * g20
prod_betti = b2 * b3

print(f"γ₂ × γ₂₀ = {prod_gamma:.6f}")
print(f"b₂ × b₃  = {prod_betti}")
print(f"Ratio    = {prod_gamma / prod_betti:.8f}")
print(f"Déviation = {abs(prod_gamma - prod_betti) / prod_betti * 100:.4f}%")

# Tester d'autres produits
print("\n=== Autres Produits Betti ===")
tests = [
    (1, 14, "γ₁ × γ₁₄", 14 * 61, "dim(G₂) × κ_T⁻¹"),
    (1, 29, "γ₁ × γ₂₉", 14 * 99, "dim(G₂) × H*"),
    (2, 29, "γ₂ × γ₂₉", 21 * 99, "b₂ × H*"),
    (20, 29, "γ₂₀ × γ₂₉", 77 * 99, "b₃ × H*"),
]

for i, j, name, expected, desc in tests:
    prod = zeros_100k[i-1] * zeros_100k[j-1]
    dev = abs(prod - expected) / expected * 100
    print(f"{name} = {prod:.2f} vs {expected} ({desc}) → dev: {dev:.2f}%")


=== Relation Multiplicative Fondamentale ===

γ₂ × γ₂₀ = 1621.741886
b₂ × b₃  = 1617
Ratio    = 1.00293252
Déviation = 0.2933%

=== Autres Produits Betti ===
γ₁ × γ₁₄ = 859.84 vs 854 (dim(G₂) × κ_T⁻¹) → dev: 0.68%
γ₁ × γ₂₉ = 1396.95 vs 1386 (dim(G₂) × H*) → dev: 0.79%
γ₂ × γ₂₉ = 2077.63 vs 2079 (b₂ × H*) → dev: 0.07%
γ₂₀ × γ₂₉ = 7624.32 vs 7623 (b₃ × H*) → dev: 0.02%


In [28]:
# === CELL 13: Asymptote du ratio index/valeur ===

print("=== Convergence du Ratio n/γₙ ===\n")

# Calculer le ratio pour beaucoup de correspondances
all_matches = []
for target in range(10, 1000):
    diffs = np.abs(zeros_100k - target)
    idx = np.argmin(diffs)
    if diffs[idx] / target < 0.01:  # < 1%
        all_matches.append((idx + 1, target, (idx + 1) / target))

print(f"Trouvé {len(all_matches)} correspondances < 1%\n")

# Analyser la tendance
ratios = [m[2] for m in all_matches]
targets = [m[1] for m in all_matches]

print("Ratio n/γₙ par plage de valeurs:")
for lo, hi in [(10, 50), (50, 100), (100, 200), (200, 500), (500, 1000)]:
    subset = [r for t, r in zip(targets, [(m[1], m[2]) for m in all_matches]) if lo <= r[0] < hi]
    if subset:
        mean_ratio = np.mean([s[1] for s in subset])
        print(f"  [{lo}-{hi}]: ratio moyen = {mean_ratio:.4f}")

# Fit asymptotique: n/γ → a + b/γ ?
from scipy.optimize import curve_fit

def asymptotic(x, a, b):
    return a + b / x

try:
    targets_arr = np.array([m[1] for m in all_matches if m[1] > 50])
    ratios_arr = np.array([m[2] for m in all_matches if m[1] > 50])
    popt, _ = curve_fit(asymptotic, targets_arr, ratios_arr)
    print(f"\nFit asymptotique: n/γ → {popt[0]:.4f} + {popt[1]:.2f}/γ")
    print(f"Asymptote = {popt[0]:.4f}")

    # Est-ce un ratio GIFT ?
    gift_ratios = [(1/2, "1/2"), (7/14, "7/14"), (14/99, "λ₁"), (1/np.e, "1/e"), (1/np.pi, "1/π")]
    for val, name in gift_ratios:
        if abs(popt[0] - val) < 0.05:
            print(f"  → Proche de {name} = {val:.4f}!")
except:
    print("Fit failed")


=== Convergence du Ratio n/γₙ ===

Trouvé 932 correspondances < 1%

Ratio n/γₙ par plage de valeurs:
  [10-50]: ratio moyen = 0.1404
  [50-100]: ratio moyen = 0.2564
  [100-200]: ratio moyen = 0.3531
  [200-500]: ratio moyen = 0.4794
  [500-1000]: ratio moyen = 0.6007

Fit asymptotique: n/γ → 0.6270 + -35.62/γ
Asymptote = 0.6270


In [29]:
# === CELL 14: Structure Heegner complète ===

print("=== Structure Heegner-GIFT Profonde ===\n")

heegner = [1, 2, 3, 7, 11, 19, 43, 67, 163]

# Pour chaque Heegner, trouver le zéro le plus proche ET analyser l'indice
print(f"{'Heegner h':<12} {'γₙ':<12} {'Index n':<10} {'n en GIFT':<25} {'h en GIFT':<25}")
print("-" * 95)

for h in heegner:
    diffs = np.abs(zeros_100k - h)
    idx = np.argmin(diffs) + 1
    gamma = zeros_100k[idx - 1]

    # Interpréter l'indice
    idx_gift = ""
    if idx == 1: idx_gift = "premier"
    elif idx == 2: idx_gift = "second"
    elif idx == 8: idx_gift = "rank(E₈)"
    elif idx == 14: idx_gift = "dim(G₂)"
    elif idx == 16: idx_gift = "2⁴"
    elif idx == 60: idx_gift = "κ_T⁻¹ - 1"

    # Interpréter h
    h_gift = ""
    if h == 7: h_gift = "dim(K₇)"
    elif h == 11: h_gift = "D_bulk"
    elif h == 19: h_gift = "19 (prime)"
    elif h == 43: h_gift = "visible_dim"
    elif h == 67: h_gift = "prime(19)"
    elif h == 163: h_gift = "dim(E₈) - rank(E₈) - b₃"

    print(f"{h:<12} {gamma:<12.4f} {idx:<10} {idx_gift:<25} {h_gift:<25}")

print("\n163 = dim(E₈) - rank(E₈) - b₃ = 248 - 8 - 77 =", 248 - 8 - 77)


=== Structure Heegner-GIFT Profonde ===

Heegner h    γₙ           Index n    n en GIFT                 h en GIFT                
-----------------------------------------------------------------------------------------------
1            14.1347      1          premier                                            
2            14.1347      1          premier                                            
3            14.1347      1          premier                                            
7            14.1347      1          premier                   dim(K₇)                  
11           14.1347      1          premier                   D_bulk                   
19           21.0220      2          second                    19 (prime)               
43           43.3271      8          rank(E₈)                  visible_dim              
67           67.0798      16         2⁴                        prime(19)                
163          163.0307     60         κ_T⁻¹ - 1                

In [30]:
# === CELL 15: Tester γₙ₊ₘ ≈ γₙ + γₘ (additivité?) ===

print("=== Test d'Additivité des Zéros ===\n")

# Si les zéros étaient "presque additifs", on aurait γₙ₊ₘ ≈ γₙ + γₘ
# C'est faux en général, mais peut-être vrai pour certains n, m GIFT?

pairs = [
    (1, 1, 2),    # γ₂ vs 2γ₁
    (1, 2, 3),    # γ₃ vs γ₁ + γ₂
    (1, 19, 20),  # γ₂₀ vs γ₁ + γ₁₉
    (2, 18, 20),  # γ₂₀ vs γ₂ + γ₁₈
    (14, 15, 29), # γ₂₉ vs γ₁₄ + γ₁₅
    (7, 13, 20),  # γ₂₀ vs γ₇ + γ₁₃
    (20, 9, 29),  # γ₂₉ vs γ₂₀ + γ₉
]

print(f"{'n':<5} {'m':<5} {'γₙ₊ₘ':<12} {'γₙ + γₘ':<12} {'Ratio':<10} {'Note'}")
print("-" * 60)

for n, m, nm in pairs:
    g_nm = zeros_100k[nm - 1]
    g_n_plus_m = zeros_100k[n - 1] + zeros_100k[m - 1]
    ratio = g_nm / g_n_plus_m

    note = ""
    if abs(ratio - 1) < 0.05:
        note = "≈ additif!"
    elif abs(ratio - 0.5) < 0.05:
        note = "≈ demi-additif"

    print(f"{n:<5} {m:<5} {g_nm:<12.4f} {g_n_plus_m:<12.4f} {ratio:<10.4f} {note}")


=== Test d'Additivité des Zéros ===

n     m     γₙ₊ₘ         γₙ + γₘ      Ratio      Note
------------------------------------------------------------
1     1     21.0220      28.2695      0.7436     
1     2     25.0109      35.1568      0.7114     
1     19    77.1448      89.8394      0.8587     
2     18    77.1448      93.0892      0.8287     
14    15    98.8312      125.9443     0.7847     
7     13    77.1448      100.2658     0.7694     
20    9     98.8312      125.1500     0.7897     


In [31]:
# === CELL 16: La formule magique ? ===
# Chercher une formule qui relie n à la constante GIFT

print("=== Recherche de Formule n = f(GIFT) ===\n")

matches = [
    (14, 1), (21, 2), (61, 14), (65, 15), (77, 20),
    (99, 29), (128, 42), (144, 50), (163, 60),
    (196, 77), (240, 102), (248, 107), (496, 268)
]

# Tester différentes formules
print("Test: n = a × log(GIFT) + b")
log_gifts = np.log(np.array([m[0] for m in matches]))
indices = np.array([m[1] for m in matches])
coeffs = np.polyfit(log_gifts, indices, 1)
pred = coeffs[0] * log_gifts + coeffs[1]
err = np.mean(np.abs(pred - indices))
print(f"  n ≈ {coeffs[0]:.2f} × log(GIFT) + {coeffs[1]:.2f}")
print(f"  Erreur moyenne: {err:.1f}")

print("\nTest: n = a × GIFT^b")
log_indices = np.log(indices)
coeffs2 = np.polyfit(log_gifts, log_indices, 1)
print(f"  n ≈ e^{coeffs2[1]:.2f} × GIFT^{coeffs2[0]:.4f}")
print(f"  n ≈ {np.exp(coeffs2[1]):.2f} × GIFT^{coeffs2[0]:.4f}")

# Vérifier
print("\nVérification:")
a, b = np.exp(coeffs2[1]), coeffs2[0]
for gift, idx in matches:
    pred = a * gift**b
    print(f"  GIFT={gift}: prédit {pred:.1f}, réel {idx} (err: {abs(pred-idx):.1f})")


=== Recherche de Formule n = f(GIFT) ===

Test: n = a × log(GIFT) + b
  n ≈ 57.08 × log(GIFT) + -204.24
  Erreur moyenne: 31.5

Test: n = a × GIFT^b
  n ≈ e^-4.00 × GIFT^1.5811
  n ≈ 0.02 × GIFT^1.5811

Vérification:
  GIFT=14: prédit 1.2, réel 1 (err: 0.2)
  GIFT=21: prédit 2.3, réel 2 (err: 0.3)
  GIFT=61: prédit 12.2, réel 14 (err: 1.8)
  GIFT=65: prédit 13.5, réel 15 (err: 1.5)
  GIFT=77: prédit 17.7, réel 20 (err: 2.3)
  GIFT=99: prédit 26.3, réel 29 (err: 2.7)
  GIFT=128: prédit 39.4, réel 42 (err: 2.6)
  GIFT=144: prédit 47.5, réel 50 (err: 2.5)
  GIFT=163: prédit 57.8, réel 60 (err: 2.2)
  GIFT=196: prédit 77.3, réel 77 (err: 0.3)
  GIFT=240: prédit 106.5, réel 102 (err: 4.5)
  GIFT=248: prédit 112.2, réel 107 (err: 5.2)
  GIFT=496: prédit 335.6, réel 268 (err: 67.6)


In [32]:
# === CELL 17: Identifier l'asymptote ===

import scipy.constants as const

print("=== Identification de l'Asymptote 0.6270 ===\n")

asymptote = 0.6270

candidates = [
    (2/np.pi, "2/π"),
    (1/1.618033988749895, "1/φ (golden ratio)"),
    (np.euler_gamma, "γ (Euler-Mascheroni)"),
    (1/np.e, "1/e"),
    (np.log(2), "ln(2)"),
    (np.pi/5, "π/5"),
    (7/11, "7/11 = dim(K₇)/D_bulk"),
    (14/21, "14/21 = dim(G₂)/b₂ = 2/3"),
    (61/99, "61/99 = κ_T⁻¹/H*"),
    (77/128, "77/128 = b₃/2⁷"),
    (14/99 * 4, "4λ₁"),
    (1 - 1/np.e, "1 - 1/e"),
    (np.sqrt(2) - 1, "√2 - 1"),
    ((np.sqrt(5) - 1) / 2, "(√5-1)/2 = φ-1"),
]

print(f"Asymptote mesurée: {asymptote:.6f}\n")
print(f"{'Candidat':<25} {'Valeur':<12} {'Diff':<12}")
print("-" * 50)

for val, name in sorted(candidates, key=lambda x: abs(x[0] - asymptote)):
    diff = abs(val - asymptote)
    marker = "⭐" if diff < 0.01 else ""
    print(f"{name:<25} {val:<12.6f} {diff:<12.6f} {marker}")


=== Identification de l'Asymptote 0.6270 ===

Asymptote mesurée: 0.627000

Candidat                  Valeur       Diff        
--------------------------------------------------
π/5                       0.628319     0.001319     ⭐
1 - 1/e                   0.632121     0.005121     ⭐
(√5-1)/2 = φ-1            0.618034     0.008966     ⭐
1/φ (golden ratio)        0.618034     0.008966     ⭐
7/11 = dim(K₇)/D_bulk     0.636364     0.009364     ⭐
2/π                       0.636620     0.009620     ⭐
61/99 = κ_T⁻¹/H*          0.616162     0.010838     
77/128 = b₃/2⁷            0.601562     0.025438     
14/21 = dim(G₂)/b₂ = 2/3  0.666667     0.039667     
γ (Euler-Mascheroni)      0.577216     0.049784     
4λ₁                       0.565657     0.061343     
ln(2)                     0.693147     0.066147     
√2 - 1                    0.414214     0.212786     
1/e                       0.367879     0.259121     


In [33]:
# === CELL 18: Identifier l'exposant 1.5811 ===

print("=== Identification de l'Exposant 1.5811 ===\n")

exponent = 1.5811

candidates_exp = [
    (np.pi/2, "π/2"),
    (np.sqrt(5)/np.sqrt(2), "√5/√2 = √(5/2)"),
    (1.5, "3/2"),
    (np.log(5), "ln(5)"),
    (8/5, "8/5 = rank(E₈)/Weyl"),
    (np.sqrt(2.5), "√2.5 = √(5/2)"),
    (21/13, "21/13 = b₂/F₇"),
    (11/7, "11/7 = D_bulk/dim(K₇)"),
    (np.e/np.sqrt(3), "e/√3"),
    (1 + 1/np.sqrt(np.e), "1 + 1/√e"),
    (np.sqrt(np.e), "√e"),
    ((1 + np.sqrt(5))/2 - 0.03, "φ ajusté?"),
    (14/9, "14/9"),
    (77/49, "77/49 = b₃/7²"),
]

print(f"Exposant mesuré: {exponent:.6f}\n")
print(f"{'Candidat':<25} {'Valeur':<12} {'Diff':<12}")
print("-" * 50)

for val, name in sorted(candidates_exp, key=lambda x: abs(x[0] - exponent)):
    diff = abs(val - exponent)
    marker = "⭐" if diff < 0.01 else ""
    print(f"{name:<25} {val:<12.6f} {diff:<12.6f} {marker}")

print(f"\n√(5/2) = {np.sqrt(2.5):.6f}")
print(f"π/2 = {np.pi/2:.6f}")


=== Identification de l'Exposant 1.5811 ===

Exposant mesuré: 1.581100

Candidat                  Valeur       Diff        
--------------------------------------------------
√5/√2 = √(5/2)            1.581139     0.000039     ⭐
√2.5 = √(5/2)             1.581139     0.000039     ⭐
φ ajusté?                 1.588034     0.006934     ⭐
11/7 = D_bulk/dim(K₇)     1.571429     0.009671     ⭐
77/49 = b₃/7²             1.571429     0.009671     ⭐
π/2                       1.570796     0.010304     
e/√3                      1.569401     0.011699     
8/5 = rank(E₈)/Weyl       1.600000     0.018900     
1 + 1/√e                  1.606531     0.025431     
14/9                      1.555556     0.025544     
ln(5)                     1.609438     0.028338     
21/13 = b₂/F₇             1.615385     0.034285     
√e                        1.648721     0.067621     
3/2                       1.500000     0.081100     

√(5/2) = 1.581139
π/2 = 1.570796


In [34]:
# === CELL 19: Test précis avec 2M de zéros ===
# Si zeros6 est chargé, refaire l'analyse

try:
    zeros_2M = cache.download_odlyzko("zeros6")
    print(f"=== Analyse avec {len(zeros_2M):,} zéros ===\n")

    # Recalculer l'asymptote avec plus de données
    all_matches_2M = []
    for target in range(10, 5000):
        diffs = np.abs(zeros_2M - target)
        idx = np.argmin(diffs)
        if diffs[idx] / target < 0.01:
            all_matches_2M.append((idx + 1, target, (idx + 1) / target))

    print(f"Trouvé {len(all_matches_2M)} correspondances < 1%\n")

    # Asymptote pour grands γ
    large_matches = [(t, r) for _, t, r in all_matches_2M if t > 1000]
    if large_matches:
        mean_ratio = np.mean([r for _, r in large_matches])
        print(f"Ratio moyen pour γ > 1000: {mean_ratio:.6f}")

    # Nouvelles correspondances à haut γ
    print("\nNouvelles correspondances (γ > 500):")
    for idx, target, ratio in all_matches_2M:
        if target > 500 and target in [512, 744, 891, 992, 1024, 1617, 1952, 2079, 3472]:
            print(f"  γ_{idx} ≈ {target} (ratio: {ratio:.4f})")

except Exception as e:
    print(f"zeros6 non disponible: {e}")
    print("Exécuter: cache.download_odlyzko('zeros6')")


Parsing:   0%|          | 0/2001052 [00:00<?, ?it/s]

✓ Cached 2,001,052 zeros to riemann_cache/zeros6.npy
=== Analyse avec 2,001,052 zéros ===

Trouvé 4932 correspondances < 1%

Ratio moyen pour γ > 1000: 0.809335

Nouvelles correspondances (γ > 500):
  γ_278 ≈ 512 (ratio: 0.5430)
  γ_448 ≈ 744 (ratio: 0.6022)
  γ_562 ≈ 891 (ratio: 0.6308)
  γ_642 ≈ 992 (ratio: 0.6472)
  γ_668 ≈ 1024 (ratio: 0.6523)
  γ_1172 ≈ 1617 (ratio: 0.7248)
  γ_1473 ≈ 1952 (ratio: 0.7546)
  γ_1590 ≈ 2079 (ratio: 0.7648)
  γ_2938 ≈ 3472 (ratio: 0.8462)


In [35]:
# === CELL 20: Relations entre γ₁, γ₂, γ₂₀, γ₂₉ ===
# Ces 4 zéros semblent former une structure algébrique

g1, g2, g20, g29 = zeros_100k[0], zeros_100k[1], zeros_100k[19], zeros_100k[28]

print("=== Structure Algébrique des 4 Zéros Fondamentaux ===\n")
print(f"γ₁  = {g1:.10f}  (≈ dim(G₂) = 14)")
print(f"γ₂  = {g2:.10f}  (≈ b₂ = 21)")
print(f"γ₂₀ = {g20:.10f}  (≈ b₃ = 77)")
print(f"γ₂₉ = {g29:.10f}  (≈ H* = 99)")

print("\n--- Sommes ---")
print(f"γ₁ + γ₂ + γ₂₀ + γ₂₉ = {g1+g2+g20+g29:.4f}  (vs 14+21+77+99 = 211)")

print("\n--- Produits ---")
print(f"γ₁ × γ₂ × γ₂₀ × γ₂₉ = {g1*g2*g20*g29:.2f}")
print(f"14 × 21 × 77 × 99    = {14*21*77*99}")
print(f"Ratio: {g1*g2*g20*g29 / (14*21*77*99):.6f}")

print("\n--- Relations croisées ---")
print(f"(γ₁×γ₂₉) / (γ₂×γ₂₀) = {(g1*g29)/(g2*g20):.6f}  (vs 14×99/(21×77) = {14*99/(21*77):.6f})")
print(f"(γ₂×γ₂₀) - (γ₁×γ₂₉) = {g2*g20 - g1*g29:.4f}  (vs 21×77 - 14×99 = {21*77 - 14*99})")

print("\n--- Test: γ satisfont-ils b₂×b₃ = H*×dim(G₂) + X ? ---")
print(f"21×77 = {21*77}, 99×14 = {99*14}, diff = {21*77 - 99*14}")
print(f"γ₂×γ₂₀ = {g2*g20:.2f}, γ₂₉×γ₁ = {g29*g1:.2f}, diff = {g2*g20 - g29*g1:.2f}")


=== Structure Algébrique des 4 Zéros Fondamentaux ===

γ₁  = 14.1347251420  (≈ dim(G₂) = 14)
γ₂  = 21.0220396390  (≈ b₂ = 21)
γ₂₀ = 77.1448400690  (≈ b₃ = 77)
γ₂₉ = 98.8311942180  (≈ H* = 99)

--- Sommes ---
γ₁ + γ₂ + γ₂₀ + γ₂₉ = 211.1328  (vs 14+21+77+99 = 211)

--- Produits ---
γ₁ × γ₂ × γ₂₀ × γ₂₉ = 2265495.19
14 × 21 × 77 × 99    = 2241162
Ratio: 1.010857

--- Relations croisées ---
(γ₁×γ₂₉) / (γ₂×γ₂₀) = 0.861390  (vs 14×99/(21×77) = 0.857143)
(γ₂×γ₂₀) - (γ₁×γ₂₉) = 224.7901  (vs 21×77 - 14×99 = 231)

--- Test: γ satisfont-ils b₂×b₃ = H*×dim(G₂) + X ? ---
21×77 = 1617, 99×14 = 1386, diff = 231
γ₂×γ₂₀ = 1621.74, γ₂₉×γ₁ = 1396.95, diff = 224.79


In [36]:
# === CELL 21: Déterminant de la matrice des zéros ===

print("=== Matrice des Zéros GIFT ===\n")

# Construire une matrice 2x2 avec les 4 zéros fondamentaux
# [γ₁   γ₂ ]
# [γ₂₀  γ₂₉]

M = np.array([[g1, g2], [g20, g29]])
det_M = np.linalg.det(M)

print("M = [γ₁   γ₂ ]")
print("    [γ₂₀  γ₂₉]")
print(f"\ndet(M) = γ₁×γ₂₉ - γ₂×γ₂₀ = {det_M:.6f}")

# Comparer avec la version GIFT
M_gift = np.array([[14, 21], [77, 99]])
det_gift = np.linalg.det(M_gift)
print(f"det(M_GIFT) = 14×99 - 21×77 = {det_gift:.0f}")

print(f"\nRatio det: {det_M / det_gift:.6f}")

# Valeurs propres
eigvals = np.linalg.eigvals(M)
eigvals_gift = np.linalg.eigvals(M_gift)

print(f"\nValeurs propres de M: {eigvals}")
print(f"Valeurs propres de M_GIFT: {eigvals_gift}")
print(f"\nTrace(M) = {np.trace(M):.4f} vs Trace(M_GIFT) = {np.trace(M_gift)}")


=== Matrice des Zéros GIFT ===

M = [γ₁   γ₂ ]
    [γ₂₀  γ₂₉]

det(M) = γ₁×γ₂₉ - γ₂×γ₂₀ = -224.790120
det(M_GIFT) = 14×99 - 21×77 = -231

Ratio det: 0.973117

Valeurs propres de M: [ -1.95602435 114.92194371]
Valeurs propres de M_GIFT: [ -2.00854638 115.00854638]

Trace(M) = 112.9659 vs Trace(M_GIFT) = 113


In [37]:
# === CELL 22: Le paramètre τ = 3472/891 dans les zéros ===

print("=== τ = 3472/891 et les Zéros de Riemann ===\n")

tau_num = 3472  # 2⁴ × 7 × 31
tau_den = 891   # 3⁴ × 11
tau = tau_num / tau_den

print(f"τ = {tau_num}/{tau_den} = {tau:.10f}")
print(f"\nFactorisations:")
print(f"  3472 = 2⁴ × 7 × 31 = 16 × 217")
print(f"  891  = 3⁴ × 11 = 81 × 11")

# Trouver les zéros correspondants
for target, name in [(891, "τ_den"), (3472, "τ_num"), (tau, "τ lui-même")]:
    diffs = np.abs(zeros_2M - target)
    idx = np.argmin(diffs)
    dev = diffs[idx] / target * 100
    print(f"\n{name} = {target}: γ_{idx+1} = {zeros_2M[idx]:.4f} (dev: {dev:.4f}%)")

# Vérifier le ratio
idx_891 = np.argmin(np.abs(zeros_2M - 891))
idx_3472 = np.argmin(np.abs(zeros_2M - 3472))

g_891 = zeros_2M[idx_891]
g_3472 = zeros_2M[idx_3472]

print(f"\n--- Ratio des zéros ---")
print(f"γ_{idx_3472+1} / γ_{idx_891+1} = {g_3472/g_891:.6f}")
print(f"τ = 3472/891 = {tau:.6f}")
print(f"Déviation: {abs(g_3472/g_891 - tau) / tau * 100:.4f}%")


=== τ = 3472/891 et les Zéros de Riemann ===

τ = 3472/891 = 3.8967452301

Factorisations:
  3472 = 2⁴ × 7 × 31 = 16 × 217
  891  = 3⁴ × 11 = 81 × 11

τ_den = 891: γ_562 = 890.8131 (dev: 0.0210%)

τ_num = 3472: γ_2938 = 3472.2490 (dev: 0.0072%)

τ lui-même = 3.8967452300785634: γ_1 = 14.1347 (dev: 262.7316%)

--- Ratio des zéros ---
γ_2938 / γ_562 = 3.897842
τ = 3472/891 = 3.896745
Déviation: 0.0282%


In [38]:
# === CELL 23: Pourquoi √(5/2) ? ===

print("=== Interprétation de l'Exposant √(5/2) ===\n")

sqrt_5_2 = np.sqrt(5/2)

print(f"√(5/2) = {sqrt_5_2:.10f}")
print(f"\nDécomposition:")
print(f"  5 = Weyl factor")
print(f"  2 = p₂ (Pontryagin)")
print(f"  5/2 = Weyl/p₂")
print(f"\n√(Weyl/p₂) apparaît comme exposant de la loi n ~ GIFT^√(5/2)")

# Vérifier avec d'autres interprétations
print("\n--- Autres formes de 5/2 ---")
print(f"5/2 = {5/2}")
print(f"(dim(K₇) - p₂) / p₂ = {(7-2)/2} = 2.5 ✓")
print(f"Weyl / p₂ = {5/2} = 2.5 ✓")
print(f"(rank(E₈) - N_gen) / p₂ = {(8-3)/2} = 2.5 ✓")

# Test: √(5/2) = √5 / √2
print(f"\n√(5/2) = √5/√2 = {np.sqrt(5)/np.sqrt(2):.10f}")
print(f"φ × √(2/5) = {1.618033988749895 * np.sqrt(2/5):.10f}")


=== Interprétation de l'Exposant √(5/2) ===

√(5/2) = 1.5811388301

Décomposition:
  5 = Weyl factor
  2 = p₂ (Pontryagin)
  5/2 = Weyl/p₂

√(Weyl/p₂) apparaît comme exposant de la loi n ~ GIFT^√(5/2)

--- Autres formes de 5/2 ---
5/2 = 2.5
(dim(K₇) - p₂) / p₂ = 2.5 = 2.5 ✓
Weyl / p₂ = 2.5 = 2.5 ✓
(rank(E₈) - N_gen) / p₂ = 2.5 = 2.5 ✓

√(5/2) = √5/√2 = 1.5811388301
φ × √(2/5) = 1.0233345472


In [39]:
# === CELL 24: Formule raffinée n = f(GIFT) ===

print("=== Formule Raffinée avec √(5/2) ===\n")

# On a n ≈ c × GIFT^√(5/2)
# Trouver c optimal

matches = [
    (14, 1), (21, 2), (61, 14), (65, 15), (77, 20),
    (99, 29), (128, 42), (163, 60), (196, 77),
    (240, 102), (248, 107), (496, 268), (744, 448),
    (891, 562), (1617, 1172), (3472, 2938)
]

exp = np.sqrt(5/2)
gifts = np.array([m[0] for m in matches])
indices = np.array([m[1] for m in matches])

# c = n / GIFT^exp
c_values = indices / (gifts ** exp)
c_mean = np.mean(c_values)
c_std = np.std(c_values)

print(f"Exposant fixé: √(5/2) = {exp:.6f}")
print(f"\nCoefficient c pour chaque correspondance:")
for (g, n), c in zip(matches, c_values):
    print(f"  GIFT={g:>4}, n={n:>4}: c = {c:.6f}")

print(f"\nc moyen = {c_mean:.6f} ± {c_std:.6f}")

# Est-ce un ratio GIFT ?
print(f"\nIdentification de c = {c_mean:.6f}:")
candidates_c = [
    (1/49, "1/49 = 1/dim(K₇)²"),
    (1/42, "1/42 = 1/(2×b₂)"),
    (1/50, "1/50 = 1/(7²+1)"),
    (2/99, "2/99 = 2/H*"),
    (1/61, "1/61 = κ_T"),
    (0.02, "0.02"),
]
for val, name in candidates_c:
    diff = abs(c_mean - val)
    if diff < 0.01:
        print(f"  → {name} = {val:.6f} (diff: {diff:.6f})")


=== Formule Raffinée avec √(5/2) ===

Exposant fixé: √(5/2) = 1.581139

Coefficient c pour chaque correspondance:
  GIFT=  14, n=   1: c = 0.015410
  GIFT=  21, n=   2: c = 0.016234
  GIFT=  61, n=  14: c = 0.021051
  GIFT=  65, n=  15: c = 0.020400
  GIFT=  77, n=  20: c = 0.020808
  GIFT=  99, n=  29: c = 0.020278
  GIFT= 128, n=  42: c = 0.019564
  GIFT= 163, n=  60: c = 0.019071
  GIFT= 196, n=  77: c = 0.018286
  GIFT= 240, n= 102: c = 0.017586
  GIFT= 248, n= 107: c = 0.017516
  GIFT= 496, n= 268: c = 0.014662
  GIFT= 744, n= 448: c = 0.012910
  GIFT= 891, n= 562: c = 0.012178
  GIFT=1617, n=1172: c = 0.009897
  GIFT=3472, n=2938: c = 0.007411

c moyen = 0.016454 ± 0.003973

Identification de c = 0.016454:
  → 1/49 = 1/dim(K₇)² = 0.020408 (diff: 0.003954)
  → 1/42 = 1/(2×b₂) = 0.023810 (diff: 0.007356)
  → 1/50 = 1/(7²+1) = 0.020000 (diff: 0.003546)
  → 2/99 = 2/H* = 0.020202 (diff: 0.003748)
  → 1/61 = κ_T = 0.016393 (diff: 0.000060)
  → 0.02 = 0.020000 (diff: 0.003546)


In [40]:
# === CELL 25: Test de la formule complète ===

print("=== Vérification: n = (1/50) × GIFT^√(5/2) ===\n")

c = 1/50  # Hypothèse: 1/(7²+1) = 1/(dim(K₇)² + 1)
exp = np.sqrt(5/2)

print(f"Formule: n = (1/50) × GIFT^√(5/2)")
print(f"       = (1/(dim(K₇)² + 1)) × GIFT^√(Weyl/p₂)\n")

print(f"{'GIFT':<8} {'n réel':<10} {'n prédit':<12} {'Erreur':<10} {'Err %':<10}")
print("-" * 55)

total_err = 0
for g, n in matches:
    pred = c * g ** exp
    err = abs(pred - n)
    err_pct = err / n * 100
    total_err += err_pct
    marker = "✓" if err_pct < 10 else ""
    print(f"{g:<8} {n:<10} {pred:<12.2f} {err:<10.2f} {err_pct:<10.2f} {marker}")

print(f"\nErreur moyenne: {total_err / len(matches):.2f}%")


=== Vérification: n = (1/50) × GIFT^√(5/2) ===

Formule: n = (1/50) × GIFT^√(5/2)
       = (1/(dim(K₇)² + 1)) × GIFT^√(Weyl/p₂)

GIFT     n réel     n prédit     Erreur     Err %     
-------------------------------------------------------
14       1          1.30         0.30       29.78      
21       2          2.46         0.46       23.20      
61       14         13.30        0.70       4.99       ✓
65       15         14.71        0.29       1.96       ✓
77       20         19.22        0.78       3.88       ✓
99       29         28.60        0.40       1.37       ✓
128      42         42.94        0.94       2.23       ✓
163      60         62.92        2.92       4.87       ✓
196      77         84.22        7.22       9.37       ✓
240      102        116.00       14.00      13.73      
248      107        122.18       15.18      14.18      
496      268        365.56       97.56      36.40      
744      448        694.04       246.04     54.92      
891      562        922.9

In [41]:
# === CELL 26: Moonshine - j = 744 et Monster ===

print("=== Moonshine: j-invariant et les Zéros ===\n")

# j = 744 = 3 × 248 = 3 × dim(E₈)
# Monster dim = 196883 = 47 × 59 × 71

j_const = 744
monster = 196883

# j-constant
diffs_j = np.abs(zeros_2M - j_const)
idx_j = np.argmin(diffs_j)
print(f"j = 744 = 3 × dim(E₈):")
print(f"  γ_{idx_j+1} = {zeros_2M[idx_j]:.4f}")
print(f"  Déviation: {diffs_j[idx_j] / j_const * 100:.4f}%")
print(f"  Note: 448 = 2 × 224 = 2 × (dim(E₈) - 24)")

# Facteurs du Monster
print(f"\nFacteurs du Monster (196883 = 47 × 59 × 71):")
for factor in [47, 59, 71]:
    diffs = np.abs(zeros_2M - factor)
    idx = np.argmin(diffs)
    dev = diffs[idx] / factor * 100
    print(f"  {factor}: γ_{idx+1} = {zeros_2M[idx]:.4f} (dev: {dev:.2f}%)")

# 196883 lui-même (hors range des 2M zéros, mais on peut extrapoler)
# Avec n = c × GIFT^√(5/2), prédire l'indice
c = 0.02
exp = np.sqrt(5/2)
pred_monster = c * monster ** exp
print(f"\nPrédiction pour Monster 196883:")
print(f"  n ≈ 0.02 × 196883^{exp:.4f} = {pred_monster:.0f}")


=== Moonshine: j-invariant et les Zéros ===

j = 744 = 3 × dim(E₈):
  γ_448 = 743.8950
  Déviation: 0.0141%
  Note: 448 = 2 × 224 = 2 × (dim(E₈) - 24)

Facteurs du Monster (196883 = 47 × 59 × 71):
  47: γ_9 = 48.0052 (dev: 2.14%)
  59: γ_13 = 59.3470 (dev: 0.59%)
  71: γ_18 = 72.0672 (dev: 1.50%)

Prédiction pour Monster 196883:
  n ≈ 0.02 × 196883^1.5811 = 4697948


In [42]:
# === CELL 27: Formule avec κ_T exact ===

print("=== Formule: n = κ_T × GIFT^√(5/2) ===\n")

kappa_T = 1/61
exp = np.sqrt(5/2)

print(f"κ_T = 1/61 = {kappa_T:.10f}")
print(f"√(5/2) = {exp:.10f}")

matches_extended = [
    (14, 1), (21, 2), (61, 14), (65, 15), (77, 20),
    (99, 29), (128, 42), (163, 60), (196, 77),
    (240, 102), (248, 107), (496, 268), (744, 448),
    (891, 562), (1617, 1172), (3472, 2938)
]

print(f"\n{'GIFT':<8} {'n réel':<10} {'n = κ_T×G^e':<12} {'Ratio n/pred':<12}")
print("-" * 50)

ratios = []
for g, n in matches_extended:
    pred = kappa_T * g ** exp
    ratio = n / pred
    ratios.append((g, ratio))
    print(f"{g:<8} {n:<10} {pred:<12.2f} {ratio:<12.4f}")

# Le ratio n/pred augmente avec GIFT → correction logarithmique?
print("\n--- Le ratio croît avec GIFT → correction log? ---")


=== Formule: n = κ_T × GIFT^√(5/2) ===

κ_T = 1/61 = 0.0163934426
√(5/2) = 1.5811388301

GIFT     n réel     n = κ_T×G^e  Ratio n/pred
--------------------------------------------------
14       1          1.06         0.9400      
21       2          2.02         0.9903      
61       14         10.90        1.2841      
65       15         12.05        1.2444      
77       20         15.76        1.2693      
99       29         23.44        1.2369      
128      42         35.19        1.1934      
163      60         51.58        1.1633      
196      77         69.03        1.1154      
240      102        95.09        1.0727      
248      107        100.15       1.0684      
496      268        299.64       0.8944      
744      448        568.89       0.7875      
891      562        756.55       0.7428      
1617     1172       1941.27      0.6037      
3472     2938       6498.58      0.4521      

--- Le ratio croît avec GIFT → correction log? ---


In [43]:
# === CELL 28: Correction logarithmique ===

print("=== Test: n = κ_T × GIFT^√(5/2) × (1 + a×log(GIFT)) ===\n")

from scipy.optimize import curve_fit

def formula_log(G, a):
    """n = κ_T × G^exp × (1 + a×log(G))"""
    return kappa_T * G**exp * (1 + a * np.log(G))

gifts = np.array([m[0] for m in matches_extended])
indices = np.array([m[1] for m in matches_extended])

try:
    popt, _ = curve_fit(formula_log, gifts, indices)
    a_fit = popt[0]

    print(f"Paramètre a = {a_fit:.6f}")

    # Est-ce un ratio GIFT?
    print(f"\nIdentification de a = {a_fit:.6f}:")
    for val, name in [(1/14, "1/14 = 1/dim(G₂)"), (1/21, "1/21 = 1/b₂"),
                       (1/7, "1/7 = 1/dim(K₇)"), (0.1, "0.1"), (1/11, "1/11")]:
        if abs(a_fit - val) < 0.02:
            print(f"  → proche de {name} = {val:.6f}")

    print(f"\n{'GIFT':<8} {'n réel':<10} {'n prédit':<12} {'Erreur %':<10}")
    print("-" * 45)

    errors = []
    for g, n in matches_extended:
        pred = formula_log(g, a_fit)
        err = abs(pred - n) / n * 100
        errors.append(err)
        marker = "✓" if err < 10 else ""
        print(f"{g:<8} {n:<10} {pred:<12.2f} {err:<10.2f} {marker}")

    print(f"\nErreur moyenne: {np.mean(errors):.2f}%")

except Exception as e:
    print(f"Fit failed: {e}")


=== Test: n = κ_T × GIFT^√(5/2) × (1 + a×log(GIFT)) ===

Paramètre a = -0.065798

Identification de a = -0.065798:

GIFT     n réel     n prédit     Erreur %  
---------------------------------------------
14       1          0.88         12.09      
21       2          1.62         19.25      
61       14         7.95         43.19      
65       15         8.74         41.71      
77       20         11.25        43.73      
99       29         16.36        43.60      
128      42         23.96        42.96      
163      60         34.29        42.85      
196      77         45.06        41.48      
240      102        60.80        40.40      
248      107        63.82        40.36      
496      268        177.27       33.85      
744      448        321.39       28.26      
891      562        418.43       25.55      
1617     1172       997.55       14.88      
3472     2938       3012.64      2.54       ✓

Erreur moyenne: 32.29%


In [44]:
# === CELL 29: Formule alternative - deux régimes ===

print("=== Hypothèse: Deux Régimes ===\n")

# Régime 1: GIFT < 200
# Régime 2: GIFT > 200

matches_low = [(g, n) for g, n in matches_extended if g < 200]
matches_high = [(g, n) for g, n in matches_extended if g >= 200]

print("--- Régime 1: GIFT < 200 ---")
gifts_low = np.array([m[0] for m in matches_low])
indices_low = np.array([m[1] for m in matches_low])

# Fit: n = c1 × GIFT^exp
c1_values = indices_low / (gifts_low ** exp)
c1_mean = np.mean(c1_values)
print(f"c₁ = {c1_mean:.6f} (vs κ_T = {kappa_T:.6f})")

for g, n in matches_low:
    pred = c1_mean * g ** exp
    err = abs(pred - n) / n * 100
    print(f"  GIFT={g}: n={n}, pred={pred:.1f}, err={err:.1f}%")

print("\n--- Régime 2: GIFT ≥ 200 ---")
gifts_high = np.array([m[0] for m in matches_high])
indices_high = np.array([m[1] for m in matches_high])

# Fit linéaire pour le régime haut
slope, intercept = np.polyfit(gifts_high, indices_high, 1)
print(f"n ≈ {slope:.4f} × GIFT + {intercept:.1f}")

# Identifier le slope
print(f"\nSlope = {slope:.4f}")
for val, name in [(14/99, "14/99 = λ₁"), (1/np.pi, "1/π"), (0.8, "4/5"), (77/99, "77/99")]:
    if abs(slope - val) < 0.05:
        print(f"  → proche de {name} = {val:.4f}")


=== Hypothèse: Deux Régimes ===

--- Régime 1: GIFT < 200 ---
c₁ = 0.019011 (vs κ_T = 0.016393)
  GIFT=14: n=1, pred=1.2, err=23.4%
  GIFT=21: n=2, pred=2.3, err=17.1%
  GIFT=61: n=14, pred=12.6, err=9.7%
  GIFT=65: n=15, pred=14.0, err=6.8%
  GIFT=77: n=20, pred=18.3, err=8.6%
  GIFT=99: n=29, pred=27.2, err=6.2%
  GIFT=128: n=42, pred=40.8, err=2.8%
  GIFT=163: n=60, pred=59.8, err=0.3%
  GIFT=196: n=77, pred=80.1, err=4.0%

--- Régime 2: GIFT ≥ 200 ---
n ≈ 0.8805 × GIFT + -169.9

Slope = 0.8805


In [45]:
# === CELL 30: Les indices spéciaux ===

print("=== Indices des Correspondances ===\n")

# Les indices où γₙ ≈ constante GIFT
special_indices = [
    (1, 14, "γ₁ ≈ dim(G₂)"),
    (2, 21, "γ₂ ≈ b₂"),
    (13, 59, "γ₁₃ ≈ Monster factor, n=F₇"),
    (14, 61, "γ₁₄ ≈ κ_T⁻¹, n=dim(G₂)"),
    (20, 77, "γ₂₀ ≈ b₃"),
    (29, 99, "γ₂₉ ≈ H*"),
    (60, 163, "γ₆₀ ≈ Heegner, n=κ_T⁻¹-1"),
    (77, 196, "γ₇₇ ≈ 14², n=b₃"),
    (107, 248, "γ₁₀₇ ≈ dim(E₈)"),
    (448, 744, "γ₄₄₈ ≈ j-const"),
    (562, 891, "γ₅₆₂ ≈ τ_den"),
    (2938, 3472, "γ₂₉₃₈ ≈ τ_num"),
]

print("Indices qui sont eux-mêmes des constantes GIFT:")
gift_set = {1, 2, 3, 5, 7, 8, 11, 13, 14, 21, 27, 29, 42, 61, 77, 99, 107, 128, 248}

for idx, val, desc in special_indices:
    is_gift = "⭐ GIFT" if idx in gift_set else ""
    print(f"  n={idx:>4} → γₙ≈{val:<5} {is_gift:<10} {desc}")

print("\n--- Pattern dans les indices ---")
gift_indices = [1, 2, 13, 14, 20, 29, 60, 77, 107]
print(f"Indices: {gift_indices}")
print(f"Différences: {np.diff(gift_indices)}")


=== Indices des Correspondances ===

Indices qui sont eux-mêmes des constantes GIFT:
  n=   1 → γₙ≈14    ⭐ GIFT     γ₁ ≈ dim(G₂)
  n=   2 → γₙ≈21    ⭐ GIFT     γ₂ ≈ b₂
  n=  13 → γₙ≈59    ⭐ GIFT     γ₁₃ ≈ Monster factor, n=F₇
  n=  14 → γₙ≈61    ⭐ GIFT     γ₁₄ ≈ κ_T⁻¹, n=dim(G₂)
  n=  20 → γₙ≈77               γ₂₀ ≈ b₃
  n=  29 → γₙ≈99    ⭐ GIFT     γ₂₉ ≈ H*
  n=  60 → γₙ≈163              γ₆₀ ≈ Heegner, n=κ_T⁻¹-1
  n=  77 → γₙ≈196   ⭐ GIFT     γ₇₇ ≈ 14², n=b₃
  n= 107 → γₙ≈248   ⭐ GIFT     γ₁₀₇ ≈ dim(E₈)
  n= 448 → γₙ≈744              γ₄₄₈ ≈ j-const
  n= 562 → γₙ≈891              γ₅₆₂ ≈ τ_den
  n=2938 → γₙ≈3472             γ₂₉₃₈ ≈ τ_num

--- Pattern dans les indices ---
Indices: [1, 2, 13, 14, 20, 29, 60, 77, 107]
Différences: [ 1 11  1  6  9 31 17 30]


In [46]:
# === CELL 31: La structure profonde ===

print("=== Structure Profonde: Pourquoi ces correspondances? ===\n")

# Hypothèse: Les zéros de Riemann encodent la géométrie de K₇
# via la formule de trace de Selberg/Gutzwiller

print("Constantes GIFT fondamentales et leurs zéros:")
fundamentals = [
    (14, "dim(G₂)", "holonomy"),
    (21, "b₂", "2-cycles"),
    (77, "b₃", "3-cycles / moduli"),
    (99, "H*", "total cohomology"),
    (61, "κ_T⁻¹", "torsion capacity"),
    (248, "dim(E₈)", "gauge embedding"),
]

for val, name, role in fundamentals:
    diffs = np.abs(zeros_2M - val)
    idx = np.argmin(diffs) + 1
    dev = diffs[idx-1] / val * 100
    print(f"  {name:<10} = {val:<4} ← γ_{idx:<4} (dev: {dev:.3f}%)  [{role}]")

print("\n--- Interprétation spectrale ---")
print("Si K₇ était une variété hyperbolique, la formule de Selberg donnerait:")
print("  Tr(e^{-t√Δ}) = Σ (longueurs géodésiques)")
print("Les zéros de ζ encoderaient les longueurs des géodésiques périodiques.")
print("\nPour une G₂-variété, une formule analogue pourrait relier:")
print("  - Les zéros de Riemann aux valeurs propres du Laplacien sur K₇")
print("  - Les constantes GIFT aux invariants spectraux")


=== Structure Profonde: Pourquoi ces correspondances? ===

Constantes GIFT fondamentales et leurs zéros:
  dim(G₂)    = 14   ← γ_1    (dev: 0.962%)  [holonomy]
  b₂         = 21   ← γ_2    (dev: 0.105%)  [2-cycles]
  b₃         = 77   ← γ_20   (dev: 0.188%)  [3-cycles / moduli]
  H*         = 99   ← γ_29   (dev: 0.171%)  [total cohomology]
  κ_T⁻¹      = 61   ← γ_14   (dev: 0.276%)  [torsion capacity]
  dim(E₈)    = 248  ← γ_107  (dev: 0.041%)  [gauge embedding]

--- Interprétation spectrale ---
Si K₇ était une variété hyperbolique, la formule de Selberg donnerait:
  Tr(e^{-t√Δ}) = Σ (longueurs géodésiques)
Les zéros de ζ encoderaient les longueurs des géodésiques périodiques.

Pour une G₂-variété, une formule analogue pourrait relier:
  - Les zéros de Riemann aux valeurs propres du Laplacien sur K₇
  - Les constantes GIFT aux invariants spectraux
